In [1]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor_bert = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

predictor_bilstm = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### MFT: Short sentences with one verb.
#### GPT-4 prompt:
Generate 50 sentences that contain exactly one verb. And specify what the verb is using a tab. Do not prepend numbers. Each sentence cannot have more than 1 verb.

Here is sample output that I want:

She loves parties.\tloves

Tell me more.\tTell

It's a beautiful day.\t's

Put your hands behind your back!\tPut

In [3]:
with open('data/predicate_identification_1.tsv','r') as infile:
    contents = infile.readlines()
    contents = [row.strip('\n').split('\t') for row in contents]
    sentences = [row[0] for row in contents]
    verbs = [row[1] for row in contents]
    print(sentences[:3])
    print(verbs[:3])    

['She reads every day.', 'He studied all night.', 'He went to the store.']
['reads', 'studied', 'went']


In [4]:
def short_sentences(predictor, sentences, verbs):
    errors = 0
    for sentence, verb in zip(sentences, verbs):
        detected_verbs = set()
        prediction = predictor.predict(sentence=sentence)
        for detected in prediction['verbs']:
            detected_verbs.add(detected['verb'])
        if verb not in detected_verbs:
            errors += 1
            print(repr((sentence, verb)))
    return errors / len(sentences)
            
bert_short_sentences_error_rate = short_sentences(predictor_bert, sentences, verbs)
print(f'(BERT) The error rate for short sentences with one verb is:', bert_short_sentences_error_rate)

bilstm_short_sentences_error_rate = short_sentences(predictor_bilstm, sentences, verbs)
print(f'(BILSTM) The error rate for short sentences with one verb is:', bilstm_short_sentences_error_rate)

(BERT) The error rate for short sentences with one verb is: 0.0
(BILSTM) The error rate for short sentences with one verb is: 0.0


### MFT: Sentences with a succession of verbs.